# preparing

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1HGDbDVoQXL50lJrrc4K77AOwmOF2b-Cc" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
!pip install transformers
!pip -q install clean-text[gpl]
!pip install bs4
!pip install sentencepiece

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 170 kB 6.8 MB/s 
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 235 kB 31.2 MB/s 
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import json
file = open("data.json", encoding="utf8")
file = json.load(file)

In [ ]:
data = []
for block in file:
  if len(block['Categories']) > 0:
    for i in range(len(block['Categories'])):
      data.append((block['Categories'][i]['Sentence'], block['Categories'], '', 0))

In [ ]:
df = pd.DataFrame(data, columns = ['Comment', 'Categories', 'Category', 'Labels'])
df

,Comment,Categories,Category,Labels
0,واقعا عالی و ب موقع رسید.,"[{'Id': 8362, 'Phrase': 'ارسال', 'Sentence': '...",,0
1,قیمتا خوب بود و سریع تحویل داده شد.,"[{'Id': 8360, 'Phrase': 'قیمت', 'Sentence': 'ق...",,0
2,قیمتا خوب بود و سریع تحویل داده شد.,"[{'Id': 8360, 'Phrase': 'قیمت', 'Sentence': 'ق...",,0
3,واقعا سایت عالی سریع مرسوله میرسونن قیمت ها عا...,"[{'Id': 8358, 'Phrase': 'ارسال', 'Sentence': '...",,0
4,واقعا سایت عالی سریع مرسوله میرسونن قیمت ها عا...,"[{'Id': 8358, 'Phrase': 'ارسال', 'Sentence': '...",,0
...,...,...,...,...
5389,وبینار شش روزه در #ایسمینار برگزار کردیم عالی بود,"[{'Id': 2723, 'Phrase': 'عمومی', 'Sentence': '...",,0
5390,از تیم ایسمینار راضی‌ام,"[{'Id': 2720, 'Phrase': 'عمومی', 'Sentence': '...",,0
5391,پشتیبانی خوب,"[{'Id': 2720, 'Phrase': 'عمومی', 'Sentence': '...",,0
5392,سایت خوب,"[{'Id': 2720, 'Phrase': 'عمومی', 'Sentence': '...",,0


In [ ]:
for i in range(len(df)):
  for j in range(len(df['Categories'][i])):
    if df['Categories'][i][j]['Phrase'] in [None,'']:
      df['Category'][i] = 'بدون کتگوری'
    if df['Categories'][i][j]['Phrase'] in ['سلامت کالا','کیفیت کالا','ضمانت کالا','اصالت کالا','تنوع کالا ','کیفیت ',' کیفیت','کیفیت']:
      df['Category'][i] = 'ضمانت،کیفیت،تنوع،مطابقت کالا'
    if df['Categories'][i][j]['Phrase'] in ['ارسال','بسته بندی','ارسال ']:
      df['Category'][i] = 'ارسال'
    if df['Categories'][i][j]['Phrase'] in ['قیمت',' قیمت','تخفیفات','قی']:
      df['Category'][i] = 'قیمت و تخفیفات'
    if df['Categories'][i][j]['Phrase'] in ['کاربری آسان','کارایی آسان','به روز بودن']:
      df['Category'][i] = 'کاربری و کارایی'
    if df['Categories'][i][j]['Phrase'] in ['مرجوعی','بازگشت وجه']:
      df['Category'][i] = 'مرجوعی و عودت وجه'
    if df['Categories'][i][j]['Phrase'] in ['عمومی','خدمات']:
      df['Category'][i] = 'خدمات - عمومی'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launc

# Data Preprocessing

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
# from PersianStemmer import PersianStemmer
from nltk.stem import WordNetLemmatizer
import string

from cleantext import clean
import codecs,re
def arabic_to_farsi(text):
    text = re.sub(r'[كﮑﮐﮏﮎﻜﻛﻚﻙ]', r'ک', text)
    text = re.sub(r'[ىىىﻴﻢﻳﻲﻱﻰىىﻯي]', r'ی', text)
    return text

def cleaning(text):
  text = str(text)
  text = re.sub(r'#', ' ', text)
  text = re.sub(r'_', ' ', text)
  text = text.replace('\u200c', ' ')
  text = re.sub(r'([ا-ی])\1{2,}', r'\1', text) # bbb+ -> b at least 2 char
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub(r'\@\w*', ' ', text)
  text = re.sub(r'\d+', ' ', text)
  text = re.sub(r'$', ' ', text)
  text = re.sub(r"[a-zA-Z0-9]+", " ",text)
  text = re.sub(r'<(.*?)>', ' ', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text)
  text=re.sub(r'[۰۱۲۳۴۵۶۷۸۹٠١١٢٣٤٥٦٧٧٨٨٩٩●]',' ',text)
  text=re.sub(r'[\·\♦\٭\\,\^\|\˝\٬\’\”\‹\▪\○¼ç½éêüəıœ™Ááàäāćíłñū©ٰٔ]',' ',text)
  text=re.sub(r'[ًٌٍَُِّْ]',' ',text)
  text=re.sub(r'[]',r' ',text)
  text=re.sub(r'[\–\—…°≈≠±≤≥\−×÷√٪→←↔↑↓\#\٫]',u' ',text) 
  text=text.replace(u'•',u' ').replace(u'ˈ',u' ').replace(u'؛',u' ').replace(u'/',u' ').replace(u'ۀ',u'هٔ').replace(u"﴿",u' ').replace(u"﴾",u' ').replace(u"'",u' ').replace(u'\\',u' ').replace(u'[',u' ').replace(u']',u' ').replace(u'?',u' ').replace(u'؟',u' ').replace(u')',u' ').replace(u'_',u' ').replace(u'(u',u' ').replace(u'}',u' ').replace(u'{',u' ').replace(u'.',u' ').replace(u'>',u' ').replace(u'<',u' ')
  text=text.replace(u'`',u' ').replace(u'\t',u' ').replace(u'=',u' ').replace(u'»',u' ').replace(u'«',u' ').replace(u'~',u' ').replace(u'!',u' ').replace(u'@',u' ').replace(u'$',u' ').replace(u',u',u' ').replace(u'%',u' ').replace(u'،',u' ').replace(u'-',u' ').replace(u';',u' ').replace(u':',u' ').replace(u'*',u' ').replace(u'"',u' ').replace(u'&',u' ').replace(u'#',u' ').replace(u'+',u' ')
  text=re.sub(r'[\n\r]{2,}',u'\n',text)
  text = re.sub(u'(\u202A|\u202B|\u202C|\u202D|\u202E|\u200F|\uFEFF|\u2003|\¬|\­)',u'\u200C', text)#حذف کارکترهای تغییرجهت
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text = re.sub(u'‌(?![ئاآأإژزرذدوؤةبپتثجچحخسشصضطظعغفقکگلمنهیيًٌٍَُِّْٰٓٔ]|[\u0900-\u097F]|ֹ)', u'', text) # در پس
  text = re.sub(u'(?<![ئبپتثجچحخسشصضطظعغفقکگلمنهیيًٌٍَُِّْٰٓٔ]|[\u0900-\u097F]|f|ֹ)‌', u'', text) # در پیش
  text=text.replace(u'­',u' ').replace(u'­',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'',u' ')
  text=text.replace(u'',u' ')
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text = re.sub(u'(\u00A0)',u' ', text).replace(u'(',u' ').replace(u')',u' ')
  text=text.replace(u'    ',u' ').replace(u'    ',u' ').replace(u'   ',u' ').replace(u'  ',u' ').replace(u'  ',u' ').replace(u'  ',u' ')
  # text=arabic_to_farsi(text)
  return text.strip()

def remove_stopwords(text):
  tokens = word_tokenize(text)
  persian_stopwords = StopWords

  results = [w for w in tokens if w not in persian_stopwords and w not in string.punctuation]
  
  return ' '.join(results)

def stemming(text):
  tokens = word_tokenize(text)
  persian_stemmer = Stemmer()

  results = [persian_stemmer.stem(w) for w in tokens]

  return ' '.join(results)

def lemmatizing(text):
  wordlist = []
  lemmatizer = WordNetLemmatizer()
  
  sentences = sent_tokenize(text)

  for sentence in sentences:
        words=word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))

  return ' '.join(wordlist)

def normalize(text):
  # result = lemmatizing(text)
  result = stemming(text)
  result = remove_stopwords(result)

  return result


In [ ]:
df['Clean_Body'] = df.Comment.apply(cleaning)

In [ ]:
label_dict = {}
labels = list(set(df.Category))
for i in range(len(list(set(df.Category)))):
  label_dict[labels[i]] = i

In [ ]:
for i in range(len(df)):
  for label in labels:
    if df['Category'][i] == label:
      df['Labels'][i] = label_dict[label]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df

,Comment,Categories,Category,Labels,Clean_Body
0,واقعا عالی و ب موقع رسید.,"[{'Id': 8362, 'Phrase': 'ارسال', 'Sentence': '...",ارسال,1,واقعا عالی و ب موقع رسید
1,قیمتا خوب بود و سریع تحویل داده شد.,"[{'Id': 8360, 'Phrase': 'قیمت', 'Sentence': 'ق...",ارسال,1,قیمتا خوب بود و سریع تحویل داده شد
2,قیمتا خوب بود و سریع تحویل داده شد.,"[{'Id': 8360, 'Phrase': 'قیمت', 'Sentence': 'ق...",ارسال,1,قیمتا خوب بود و سریع تحویل داده شد
3,واقعا سایت عالی سریع مرسوله میرسونن قیمت ها عا...,"[{'Id': 8358, 'Phrase': 'ارسال', 'Sentence': '...",قیمت و تخفیفات,4,واقعا سایت عالی سریع مرسوله میرسونن قیمت ها عا...
4,واقعا سایت عالی سریع مرسوله میرسونن قیمت ها عا...,"[{'Id': 8358, 'Phrase': 'ارسال', 'Sentence': '...",قیمت و تخفیفات,4,واقعا سایت عالی سریع مرسوله میرسونن قیمت ها عا...
...,...,...,...,...,...
5389,وبینار شش روزه در #ایسمینار برگزار کردیم عالی بود,"[{'Id': 2723, 'Phrase': 'عمومی', 'Sentence': '...",خدمات - عمومی,5,وبینار شش روزه در ایسمینار برگزار کردیم عالی بود
5390,از تیم ایسمینار راضی‌ام,"[{'Id': 2720, 'Phrase': 'عمومی', 'Sentence': '...",کاربری و کارایی,3,از تیم ایسمینار راضی ام
5391,پشتیبانی خوب,"[{'Id': 2720, 'Phrase': 'عمومی', 'Sentence': '...",کاربری و کارایی,3,پشتیبانی خوب
5392,سایت خوب,"[{'Id': 2720, 'Phrase': 'عمومی', 'Sentence': '...",کاربری و کارایی,3,سایت خوب


# Model

In [ ]:
import time, sys
import torch
from torch import optim
from torch.utils.data import DataLoader
from IPython.display import clear_output
import torch.nn.functional as F

class DigiMagDs(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

class TorchTrainer:
    def __init__(self, model, train_dl, valid_dl, test_dl, optimizer, scheduler):
        self.model = model
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.test_dl = test_dl
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.loss_history = []

    def fit(self, num_epochs):
        clear_output()
        valid_acc = 0
        for epoch in range(num_epochs):
            print('Epoch %2d/%2d' % (epoch + 1, num_epochs))
            print('-' * 20)
            t0 = time.time()
            train_acc = self.train_model()
            valid_acc, _,_= self.valid_model()
            time_elapsed = time.time() - t0
            print('\n Metrics: | train_acc: %.3f | valid_acc: %.3f |' % (train_acc[0], valid_acc[0]))
            print('\n  Epoch complete in: %.0fm %.0fs \n' % (time_elapsed // 60, time_elapsed % 60))
        return

    def train_model(self):
        self.model.train()
        N = len(self.train_dl.dataset)
        step = N // self.train_dl.batch_size
        avg_loss = 0.0
        avg_acc = 0.0
        for i, batch in enumerate(self.train_dl):
            self.optimizer.zero_grad()
            # forward
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
            predictions = torch.argmax(outputs['logits'], dim=1)
            # loss
            loss = outputs['loss']
            # backward
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            # statistics of model training and print
            avg_loss = (avg_loss * i + loss) / (i + 1)
            avg_acc += torch.sum(predictions == labels)
            self.loss_history.append(avg_loss)
            sys.stdout.flush()
            sys.stdout.write("\r  Train_Step: %d/%d | runing_loss: %.4f" % (i + 1, step, avg_loss))

        sys.stdout.flush()
        return torch.tensor([avg_acc]) / N

    def valid_model(self):
        print()
        self.model.eval()
        N = len(self.valid_dl.dataset)
        step = N // self.valid_dl.batch_size
        avg_loss = 0.0
        avg_acc = 0.0
        Predictions = []
        Labels = []
        with torch.no_grad():
            for i, batch in enumerate(self.valid_dl):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                predictions = torch.argmax(outputs['logits'], dim=1)
                loss = outputs['loss']
                avg_loss = (avg_loss * i + loss) / (i + 1)
                avg_acc += torch.sum(predictions == labels)
                Predictions = Predictions + predictions.tolist()
                Labels = Labels + labels.tolist()
                sys.stdout.flush()
                sys.stdout.write("\r  Valid_Step: %d/%d | runing_loss: %.4f" % (i, step, avg_loss))

        sys.stdout.flush()
        return torch.tensor([avg_acc]) /N ,Predictions,Labels

    def test_model(self):
        print()
        self.model.eval()
        N = len(self.test_dl.dataset)
        step = N // self.test_dl.batch_size
        avg_loss = 0.0
        avg_acc = 0.0
        Predictions = []
        Labels = []
        with torch.no_grad():
            for i, batch in enumerate(self.test_dl):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                predictions = torch.argmax(outputs['logits'], dim=1)
                loss = outputs['loss']
                avg_loss = (avg_loss * i + loss) / (i + 1)
                avg_acc += torch.sum(predictions == labels)
                Predictions = Predictions + predictions.tolist()
                Labels = Labels + labels.tolist()
                sys.stdout.flush()
                sys.stdout.write("\r  Valid_Step: %d/%d | runing_loss: %.4f" % (i, step, avg_loss))

        sys.stdout.flush()
        return torch.tensor([avg_acc]) /N ,Predictions,Labels

    def test_model_probs(self):
        print()
        self.model.eval()
        N = len(self.test_dl.dataset)
        step = N // self.test_dl.batch_size
        avg_loss = 0.0
        avg_acc = 0.0
        Predictions = []
        Probabilities = []
        Labels = []
        with torch.no_grad():
            for i, batch in enumerate(self.test_dl):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                predictions = torch.argmax(outputs['logits'], dim=1)
                probabilities = F.softmax(outputs['logits'], dim=-1)
                loss = outputs['loss']
                avg_loss = (avg_loss * i + loss) / (i + 1)
                avg_acc += torch.sum(predictions == labels)
                Predictions = Predictions + predictions.tolist()
                probabilities = [max(x) for x in probabilities.tolist()]
                Probabilities = Probabilities + probabilities
                Labels = Labels + labels.tolist()
                sys.stdout.flush()
                sys.stdout.write("\r  Valid_Step: %d/%d | runing_loss: %.4f" % (i, step, avg_loss))

        sys.stdout.flush()
        return torch.tensor([avg_acc]) /N ,Probabilities,Labels

In [ ]:
batch_size = 32
max_len = 120
class_number = len(list(set(df.Category)))
RANDOM_SEED = 42
model_name = "HooshvareLab/albert-fa-zwnj-base-v2"

In [ ]:
from sklearn.model_selection import train_test_split

train_df,test_df = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
train_df,valid_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
train_df.shape, valid_df.shape, test_df.shape

((4368, 5), (486, 5), (540, 5))

In [ ]:
from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained(model_name)
train_encodings = tokenizer(train_df.Clean_Body.tolist(), max_length=max_len, truncation=True, padding=True)
print('train tokenized')
test_encodings = tokenizer(test_df.Clean_Body.tolist(), max_length=max_len, truncation=True, padding=True)
print('test tokenized')
valid_encodings = tokenizer(valid_df.Clean_Body.tolist(), max_length=max_len, truncation=True, padding=True)
print('valid tokenized')

train tokenized
test tokenized
valid tokenized


In [ ]:
from transformers import AutoConfig

# create a dict for classes
label2id = {label: i for i, label in enumerate(range(class_number))}
id2label = {v: k for k, v in label2id.items()}
config = AutoConfig.from_pretrained(model_name,**{'label2id': label2id, 'id2label': id2label})

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
id2label: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}


In [ ]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from transformers import AlbertForSequenceClassification, AutoModel
# set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class DigiMagDs(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# create datasets
train_ds = DigiMagDs(train_encodings, train_df.Labels)
valid_ds = DigiMagDs(valid_encodings, valid_df.Labels)
test_ds = DigiMagDs(test_encodings, test_df.Labels)


# create Dataloders
# train_ds = PandasDataset(train_ds)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
# valid_ds = PandasDataset(valid_ds)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=True, num_workers=2)
# test_ds = PandasDataset(test_ds)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
model = AlbertForSequenceClassification.from_pretrained(model_name, config=config)
model.to(device)
# AdamW is just adam with fixing on weight decay (don't worry about it)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
num_epoch = 6
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epoch *len(train_dl))

Some weights of the model checkpoint at HooshvareLab/albert-fa-zwnj-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/albert-fa-zwnj-base-v2 and are newly initialized: ['albe

In [ ]:
trainer = TorchTrainer(model, train_dl, valid_dl, test_dl, optimizer=optimizer, scheduler = scheduler)
trainer.fit(num_epochs=num_epoch)

Epoch  1/ 6
--------------------
  Train_Step: 137/136 | runing_loss: 1.6461
  Valid_Step: 15/15 | runing_loss: 1.5578
 Metrics: | train_acc: 0.392 | valid_acc: 0.426 |

  Epoch complete in: 3m 1s 

Epoch  2/ 6
--------------------
  Train_Step: 137/136 | runing_loss: 1.4026
  Valid_Step: 15/15 | runing_loss: 1.3614
 Metrics: | train_acc: 0.502 | valid_acc: 0.533 |

  Epoch complete in: 3m 1s 

Epoch  3/ 6
--------------------
  Train_Step: 137/136 | runing_loss: 1.1018
  Valid_Step: 15/15 | runing_loss: 1.2358
 Metrics: | train_acc: 0.629 | valid_acc: 0.576 |

  Epoch complete in: 3m 1s 

Epoch  4/ 6
--------------------
  Train_Step: 137/136 | runing_loss: 0.7139
  Valid_Step: 15/15 | runing_loss: 1.1800
 Metrics: | train_acc: 0.765 | valid_acc: 0.628 |

  Epoch complete in: 3m 1s 

Epoch  5/ 6
--------------------
  Train_Step: 137/136 | runing_loss: 0.3800
  Valid_Step: 15/15 | runing_loss: 1.2141
 Metrics: | train_acc: 0.893 | valid_acc: 0.654 |

  Epoch complete in: 3m 1s 

Epoch

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
Accuracy, y_pred, y_true = trainer.test_model()
 
print(Accuracy)
print(precision_recall_fscore_support(y_true, y_pred, average='micro'))
print(precision_recall_fscore_support(y_true, y_pred, average='macro'))
print(precision_recall_fscore_support(y_true, y_pred, average='weighted'))


  Valid_Step: 16/16 | runing_loss: 1.2297tensor([0.6389])
(0.6388888888888888, 0.6388888888888888, 0.6388888888888888, None)
(0.5447158493503664, 0.5260592875374054, 0.5328243151813717, None)
(0.6364853755806347, 0.6388888888888888, 0.636712359737715, None)


In [ ]:
Accuracy, y_pred, y_true = trainer.test_model_probs()


  Valid_Step: 16/16 | runing_loss: 1.2269

In [ ]:
from sklearn.metrics import f1_score
from numpy import arange

thresholds = np.arange(0.1,0.5,0.01)
probs = y_pred
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')
 
scores = [f1_score(y_true, to_labels(probs, t), average='weighted') for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.480, F-Score=0.09675


In [ ]:
scores[ix]

0.09675093783620917